## Pull in UDFs

In [ ]:
%run nb_udfs

## Run Dataflows

In [ ]:
workspace = 'Fabric of Middle-Earth' #have to escape the & symbol using %26% 
lakehouse = 'lh_the_hoard'
dataflow_table = 'dimDataflows'


In [ ]:
#get a list of all dataflows and load to a table in the lakehouse
response = _base_api(
        request=f"/v1.0/myorg/admin/dataflows",
        method="get"
    )
df_dataflows = pd.json_normalize(response.json()['value'])
df_dataflows = spark.createDataFrame(df_dataflows)
df_dataflows = df_dataflows.drop("users")
#df_dataflows.printSchema()
#print(df_dataflows.columns)


In [ ]:
#creates a slowly changing dimension so we can keep an eye on any deleted dataflows
dataflowPath = udf_GetFilePath(workspace, lakehouse, dataflow_table)
#print(dataflowPath)
naturalKeyColumnList = ['objectId']
primaryKeyColumnName = "tableId"
returnVal = udf_UpsertDimension(df_dataflows,2,dataflowPath,naturalKeyColumnList,primaryKeyColumnName,True)
print(returnVal)